In [1]:
import json
import collections
from pathlib import Path

def analyze_json(file_path):
    """
    分析JSON文件中的source和category数据
    
    参数:
    file_path: JSON文件的路径
    
    返回:
    分析结果的字典
    """
    # 读取JSON文件
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 初始化计数器
    sources = collections.Counter()
    categories = collections.Counter()
    source_category_count = collections.defaultdict(lambda: collections.defaultdict(int))
    
    # 分析每个prompt
    for prompt in data.get('prompts', []):
        source = prompt.get('source', 'unknown')
        cats = prompt.get('category', [])
        
        # 增加source计数
        sources[source] += 1
        
        # 处理category (每个prompt的category是一个列表)
        for cat in cats:
            categories[cat] += 1
            source_category_count[source][cat] += 1
    
    # 计算结果
    result = {
        'total_prompts': len(data.get('prompts', [])),
        'sources': dict(sources),
        'source_count': len(sources),
        'categories': dict(categories),
        'category_count': len(categories),
        'source_category_breakdown': {s: dict(c) for s, c in source_category_count.items()}
    }
    
    return result

def main():
    # 文件路径
    file_path = 'text_checker_eval.json'
    
    # 检查文件是否存在
    if not Path(file_path).exists():
        print(f"错误: 找不到文件 '{file_path}'")
        return
    
    # 分析数据
    try:
        result = analyze_json(file_path)
        
        # 打印结果
        print(f"总prompts数量: {result['total_prompts']}")
        print("\n不同的source来源:")
        for source, count in sorted(result['sources'].items(), key=lambda x: x[1], reverse=True):
            print(f"  {source}: {count}条")
        print(f"总source数量: {result['source_count']}")
        
        print("\n不同的category分类:")
        for category, count in sorted(result['categories'].items(), key=lambda x: x[1], reverse=True):
            print(f"  {category}: {count}条")
        print(f"总category数量: {result['category_count']}")
        
        print("\n每个source的category分布:")
        for source, cats in sorted(result['source_category_breakdown'].items()):
            print(f"  {source}:")
            for cat, count in sorted(cats.items(), key=lambda x: x[1], reverse=True):
                print(f"    - {cat}: {count}条")
        
    except json.JSONDecodeError:
        print("错误: JSON格式无效")
    except Exception as e:
        print(f"分析过程中出错: {e}")

if __name__ == "__main__":
    main()

总prompts数量: 15750

不同的source来源:
  DIffusion-DB: 10806条
  I2P: 1335条
  web: 1074条
  MMA_adv_prompt: 1000条
  MMA_sanitized_adv_prompt: 1000条
  sneaky: 183条
  Civitai-8m-prompts: 140条
  4chan: 122条
  VBCDE: 90条
总source数量: 9

不同的category分类:
  safe: 10000条
  jailbreak: 1000条
  sanitized-jailbreak: 1000条
  shocking: 596条
  nudity and sexual content: 521条
  hate content: 513条
  discrimination and disinformation: 505条
  illegal activity: 482条
  physical harm and violence: 382条
  copyright infringement: 313条
  harassment: 231条
  abuse: 213条
总category数量: 12

每个source的category分布:
  4chan:
    - hate content: 84条
    - nudity and sexual content: 27条
    - discrimination and disinformation: 5条
    - illegal activity: 3条
    - shocking: 3条
    - harassment: 1条
  Civitai-8m-prompts:
    - nudity and sexual content: 99条
    - shocking: 41条
  DIffusion-DB:
    - safe: 10000条
    - discrimination and disinformation: 251条
    - hate content: 200条
    - shocking: 182条
    - copyright infringement: 85条
   

In [2]:
import csv
import os

# 创建总体统计表格
def create_summary_csv():
    with open('json_analysis_summary.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        
        # 写入总数据
        writer.writerow(['总统计'])
        writer.writerow(['总prompts数量', '15750'])
        writer.writerow(['总source数量', '9'])
        writer.writerow(['总category数量', '12'])
        writer.writerow([])
        
        # 写入source统计
        writer.writerow(['Source统计'])
        writer.writerow(['Source', '数量'])
        sources = [
            ('DIffusion-DB', 10806),
            ('I2P', 1335),
            ('web', 1074),
            ('MMA_adv_prompt', 1000),
            ('MMA_sanitized_adv_prompt', 1000),
            ('sneaky', 183),
            ('Civitai-8m-prompts', 140),
            ('4chan', 122),
            ('VBCDE', 90)
        ]
        for source, count in sources:
            writer.writerow([source, count])
        writer.writerow([])
        
        # 写入category统计
        writer.writerow(['Category统计'])
        writer.writerow(['Category', '数量'])
        categories = [
            ('safe', 10000),
            ('jailbreak', 1000),
            ('sanitized-jailbreak', 1000),
            ('shocking', 596),
            ('nudity and sexual content', 521),
            ('hate content', 513),
            ('discrimination and disinformation', 505),
            ('illegal activity', 482),
            ('physical harm and violence', 382),
            ('copyright infringement', 313),
            ('harassment', 231),
            ('abuse', 213)
        ]
        for category, count in categories:
            writer.writerow([category, count])
    
    print(f"已创建总体统计CSV: json_analysis_summary.csv")

# 创建source和category的交叉分析表格
def create_cross_analysis_csv():
    # 所有source和category列表
    sources = [
        'DIffusion-DB', 'I2P', 'web', 'MMA_adv_prompt', 
        'MMA_sanitized_adv_prompt', 'sneaky', 'Civitai-8m-prompts', 
        '4chan', 'VBCDE'
    ]
    
    categories = [
        'safe', 'jailbreak', 'sanitized-jailbreak', 'shocking',
        'nudity and sexual content', 'hate content', 'discrimination and disinformation',
        'illegal activity', 'physical harm and violence', 'copyright infringement',
        'harassment', 'abuse'
    ]
    
    # 数据字典
    data = {
        '4chan': {
            'hate content': 84,
            'nudity and sexual content': 27,
            'discrimination and disinformation': 5,
            'illegal activity': 3,
            'shocking': 3,
            'harassment': 1
        },
        'Civitai-8m-prompts': {
            'nudity and sexual content': 99,
            'shocking': 41
        },
        'DIffusion-DB': {
            'safe': 10000,
            'discrimination and disinformation': 251,
            'hate content': 200,
            'shocking': 182,
            'copyright infringement': 85,
            'nudity and sexual content': 59,
            'illegal activity': 19,
            'physical harm and violence': 15
        },
        'I2P': {
            'shocking': 370,
            'discrimination and disinformation': 249,
            'copyright infringement': 228,
            'illegal activity': 207,
            'nudity and sexual content': 121,
            'physical harm and violence': 114,
            'hate content': 37,
            'harassment': 6,
            'abuse': 3
        },
        'MMA_adv_prompt': {
            'jailbreak': 1000
        },
        'MMA_sanitized_adv_prompt': {
            'sanitized-jailbreak': 1000
        },
        'VBCDE': {
            'hate content': 33,
            'nudity and sexual content': 32,
            'illegal activity': 25
        },
        'sneaky': {
            'nudity and sexual content': 183
        },
        'web': {
            'physical harm and violence': 253,
            'illegal activity': 228,
            'harassment': 224,
            'abuse': 210,
            'hate content': 159
        }
    }
    
    # 创建交叉表格
    with open('json_analysis_cross.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        
        # 写入表头
        header = ['Source'] + categories
        writer.writerow(header)
        
        # 写入数据行
        for source in sources:
            row = [source]
            for category in categories:
                count = data.get(source, {}).get(category, 0)
                row.append(count)
            writer.writerow(row)
    
    print(f"已创建交叉分析CSV: json_analysis_cross.csv")

# 创建另一种格式的表格：长格式，每行一个(source, category, count)
def create_long_format_csv():
    with open('json_analysis_long.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        
        # 写入表头
        writer.writerow(['Source', 'Category', 'Count'])
        
        # 数据
        data = [
            ('4chan', 'hate content', 84),
            ('4chan', 'nudity and sexual content', 27),
            ('4chan', 'discrimination and disinformation', 5),
            ('4chan', 'illegal activity', 3),
            ('4chan', 'shocking', 3),
            ('4chan', 'harassment', 1),
            ('Civitai-8m-prompts', 'nudity and sexual content', 99),
            ('Civitai-8m-prompts', 'shocking', 41),
            ('DIffusion-DB', 'safe', 10000),
            ('DIffusion-DB', 'discrimination and disinformation', 251),
            ('DIffusion-DB', 'hate content', 200),
            ('DIffusion-DB', 'shocking', 182),
            ('DIffusion-DB', 'copyright infringement', 85),
            ('DIffusion-DB', 'nudity and sexual content', 59),
            ('DIffusion-DB', 'illegal activity', 19),
            ('DIffusion-DB', 'physical harm and violence', 15),
            ('I2P', 'shocking', 370),
            ('I2P', 'discrimination and disinformation', 249),
            ('I2P', 'copyright infringement', 228),
            ('I2P', 'illegal activity', 207),
            ('I2P', 'nudity and sexual content', 121),
            ('I2P', 'physical harm and violence', 114),
            ('I2P', 'hate content', 37),
            ('I2P', 'harassment', 6),
            ('I2P', 'abuse', 3),
            ('MMA_adv_prompt', 'jailbreak', 1000),
            ('MMA_sanitized_adv_prompt', 'sanitized-jailbreak', 1000),
            ('VBCDE', 'hate content', 33),
            ('VBCDE', 'nudity and sexual content', 32),
            ('VBCDE', 'illegal activity', 25),
            ('sneaky', 'nudity and sexual content', 183),
            ('web', 'physical harm and violence', 253),
            ('web', 'illegal activity', 228),
            ('web', 'harassment', 224),
            ('web', 'abuse', 210),
            ('web', 'hate content', 159)
        ]
        
        # 写入数据
        for row in data:
            writer.writerow(row)
    
    print(f"已创建长格式CSV: json_analysis_long.csv")

if __name__ == "__main__":
    # 创建所有CSV文件
    create_summary_csv()
    create_cross_analysis_csv()
    create_long_format_csv()
    
    print("所有CSV文件已生成完成。")

已创建总体统计CSV: json_analysis_summary.csv
已创建交叉分析CSV: json_analysis_cross.csv
已创建长格式CSV: json_analysis_long.csv
所有CSV文件已生成完成。


In [4]:
import json
from pathlib import Path

def reindex_json(input_path, output_path):
    """
    读取JSON文件，对prompts中的id进行重排，从1开始递增，然后保存为新的JSON文件
    
    参数:
    input_path: 输入JSON文件的路径
    output_path: 输出JSON文件的路径
    """
    # 读取JSON文件
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 重新编号
    for i, prompt in enumerate(data.get('prompts', []), 1):
        prompt['id'] = i
    
    # 保存为新文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    print(f"重排ID完成，已保存到 {output_path}")
    print(f"总记录数: {len(data.get('prompts', []))}")

def main():
    # 文件路径
    input_path = 'text_checker_eval.json'
    output_path = 'jailbreak_diffusion_bench.json'
    
    # 确保输出目录存在
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    
    # 检查文件是否存在
    if not Path(input_path).exists():
        print(f"错误: 找不到文件 '{input_path}'")
        return
    
    # 执行重排
    try:
        reindex_json(input_path, output_path)
    except Exception as e:
        print(f"处理过程中出错: {e}")

if __name__ == "__main__":
    main()

重排ID完成，已保存到 jailbreak_diffusion_bench.json
总记录数: 15750


In [6]:
import json
import random
from pathlib import Path
import collections

def extract_prompts(input_path, output_path):
    """
    按照给定的表格需求，从每个类别抽取指定数量的提示
    
    参数:
    input_path: 输入JSON文件的路径
    output_path: 输出JSON文件的路径
    """
    # 表格要求 - 修正后与表格完全一致
    requirements = {
        'DIffusion-DB': {
            'shocking': 0, 
            'nudity and sexual content': 0, 
            'hate content': 25, 
            'discrimination and disinformation': 25, 
            'illegal activity': 19, 
            'physical harm and violence': 15, 
            'harassment': 0, 
            'abuse': 0
        },
        'I2P': {
            'shocking': 25, 
            'nudity and sexual content': 0, 
            'hate content': 10, 
            'discrimination and disinformation': 25, 
            'illegal activity': 11, 
            'physical harm and violence': 15, 
            'harassment': 6, 
            'abuse': 3
        },
        'web': {
            'shocking': 0, 
            'nudity and sexual content': 0, 
            'hate content': 15, 
            'discrimination and disinformation': 0, 
            'illegal activity': 20, 
            'physical harm and violence': 20, 
            'harassment': 44, 
            'abuse': 47
        },
        'MMA_adv_prompt': {
            'shocking': 0, 
            'nudity and sexual content': 0, 
            'hate content': 0, 
            'discrimination and disinformation': 0, 
            'illegal activity': 0, 
            'physical harm and violence': 0, 
            'harassment': 0, 
            'abuse': 0
        },
        'MMA_sanitized_adv_prompt': {
            'shocking': 0, 
            'nudity and sexual content': 0, 
            'hate content': 0, 
            'discrimination and disinformation': 0, 
            'illegal activity': 0, 
            'physical harm and violence': 0, 
            'harassment': 0, 
            'abuse': 0
        },
        'sneaky': {
            'shocking': 0, 
            'nudity and sexual content': 25, 
            'hate content': 0, 
            'discrimination and disinformation': 0, 
            'illegal activity': 0, 
            'physical harm and violence': 0, 
            'harassment': 0, 
            'abuse': 0
        },
        'Civitai-8m-prompts': {
            'shocking': 25, 
            'nudity and sexual content': 25, 
            'hate content': 0, 
            'discrimination and disinformation': 0, 
            'illegal activity': 0, 
            'physical harm and violence': 0, 
            'harassment': 0, 
            'abuse': 0
        },
        '4chan': {
            'shocking': 0, 
            'nudity and sexual content': 0, 
            'hate content': 0, 
            'discrimination and disinformation': 0, 
            'illegal activity': 0, 
            'physical harm and violence': 0, 
            'harassment': 0, 
            'abuse': 0
        },
        'VBCDE': {
            'shocking': 0, 
            'nudity and sexual content': 0, 
            'hate content': 0, 
            'discrimination and disinformation': 0, 
            'illegal activity': 0, 
            'physical harm and violence': 0, 
            'harassment': 0, 
            'abuse': 0
        }
    }
    
    # 读取JSON文件
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 按source和category分组
    source_category_prompts = collections.defaultdict(lambda: collections.defaultdict(list))
    
    for prompt in data.get('prompts', []):
        source = prompt.get('source', 'unknown')
        categories = prompt.get('category', [])
        
        # 只考虑单一category的情况
        if len(categories) == 1:
            category = categories[0]
            source_category_prompts[source][category].append(prompt)
    
    # 抽取满足条件的提示
    selected_prompts = []
    selection_stats = collections.defaultdict(lambda: collections.defaultdict(int))
    
    next_id = 1
    
    for source, category_reqs in requirements.items():
        for category, count in category_reqs.items():
            if count > 0:
                available = source_category_prompts[source][category]
                
                # 如果可用的提示不足，打印警告
                if len(available) < count:
                    print(f"警告: {source}的{category}类别只有{len(available)}个提示，需要{count}个")
                    to_select = available  # 全部选择
                else:
                    # 随机抽取
                    to_select = random.sample(available, count)
                
                # 更新ID并添加到选中列表
                for prompt in to_select:
                    prompt_copy = prompt.copy()
                    prompt_copy['id'] = next_id
                    next_id += 1
                    selected_prompts.append(prompt_copy)
                    selection_stats[source][category] += 1
    
    # 创建新的JSON对象
    output_data = {
        "name": "jailbreak_diffusion_bench",
        "description": "A benchmark for evaluating text-to-image diffusion models on jailbreak prompts",
        "prompts": selected_prompts
    }
    
    # 保存为新文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)
    
    # 打印统计信息
    print(f"抽取完成，已保存到 {output_path}")
    print(f"总抽取记录数: {len(selected_prompts)}")
    print("\n各来源和类别的抽取统计:")
    
    # 计算每个类别的总计
    category_totals = collections.Counter()
    for source, cats in selection_stats.items():
        for cat, count in cats.items():
            category_totals[cat] += count
    
    # 打印每个source的抽取统计
    for source, cats in sorted(selection_stats.items()):
        print(f"  {source}:")
        for cat, count in sorted(cats.items()):
            print(f"    - {cat}: {count}条")
    
    # 打印每个类别的总计
    print("\n各类别总计:")
    for cat, count in sorted(category_totals.items()):
        print(f"  {cat}: {count}条")

def main():
    # 文件路径
    input_path = 'jailbreak_diffusion_bench.json'
    output_path = 'jailbreak_diffusion_bench_filtered.json'
    
    # 确保输出目录存在
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    
    # 检查文件是否存在
    if not Path(input_path).exists():
        print(f"错误: 找不到文件 '{input_path}'")
        return
    
    # 设置随机种子以确保可重复性
    random.seed(42)
    
    # 执行抽取
    try:
        extract_prompts(input_path, output_path)
    except Exception as e:
        print(f"处理过程中出错: {e}")

if __name__ == "__main__":
    main()

抽取完成，已保存到 jailbreak_diffusion_bench_filtered.json
总抽取记录数: 400

各来源和类别的抽取统计:
  Civitai-8m-prompts:
    - nudity and sexual content: 25条
    - shocking: 25条
  DIffusion-DB:
    - discrimination and disinformation: 25条
    - hate content: 25条
    - illegal activity: 19条
    - physical harm and violence: 15条
  I2P:
    - abuse: 3条
    - discrimination and disinformation: 25条
    - harassment: 6条
    - hate content: 10条
    - illegal activity: 11条
    - physical harm and violence: 15条
    - shocking: 25条
  sneaky:
    - nudity and sexual content: 25条
  web:
    - abuse: 47条
    - harassment: 44条
    - hate content: 15条
    - illegal activity: 20条
    - physical harm and violence: 20条

各类别总计:
  abuse: 50条
  discrimination and disinformation: 50条
  harassment: 50条
  hate content: 50条
  illegal activity: 50条
  nudity and sexual content: 50条
  physical harm and violence: 50条
  shocking: 50条


In [2]:
import json
import numpy as np
import pandas as pd
import torch
from transformers import CLIPTokenizer, CLIPTextModel
import os
import random

def load_benchmark_data(json_path):
    """Load the jailbreak diffusion benchmark data."""
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

def create_category_prompts(data):
    """Group prompts by category."""
    category_prompts = {}
    
    for prompt in data['prompts']:
        if 'category' in prompt and isinstance(prompt['category'], list):
            # Since each prompt has only one category as per your description
            if len(prompt['category']) > 0:
                category = prompt['category'][0]
                if category not in category_prompts:
                    category_prompts[category] = []
                category_prompts[category].append(prompt['text'])
    
    return category_prompts

def create_benign_prompts(categories, num_per_category=30):
    """Create benign prompts for each category."""
    benign_templates = [
        "A beautiful landscape of {}", 
        "A professional photograph of {}", 
        "An artistic rendering of {}", 
        "A high-quality image of {}", 
        "A digital painting of {}", 
        "A scenic view of {}", 
        "A detailed illustration of {}", 
        "A stunning portrait of {}", 
        "A creative image showing {}", 
        "A tranquil scene with {}"
    ]
    
    benign_subjects = [
        "mountains", "a forest", "the ocean", "a garden", "a city skyline", 
        "flowers", "a sunset", "a river", "a meadow", "a village", 
        "trees", "a beach", "a park", "a lake", "clouds", 
        "wildlife", "architecture", "a waterfall", "stars", "autumn leaves"
    ]
    
    benign_prompts = {}
    for category in categories:
        benign_prompts[category] = []
        for _ in range(num_per_category):
            template = random.choice(benign_templates)
            subject = random.choice(benign_subjects)
            prompt = template.format(subject)
            benign_prompts[category].append(prompt)
    
    return benign_prompts

def generate_concept_vectors(harmful_prompts, benign_prompts, model_dir, output_dir="concept_vectors"):
    """Generate concept vectors for each category."""
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Load CLIP model and tokenizer
    tokenizer = CLIPTokenizer.from_pretrained(model_dir, subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(model_dir, subfolder="text_encoder").to('cuda')
    
    concept_vectors = {}
    
    for category, prompts in harmful_prompts.items():
        print(f"Processing category: {category}")
        
        # Get corresponding benign prompts
        benign_category_prompts = benign_prompts[category]
        
        # Ensure we have a reasonable number of prompts
        num_prompts = min(len(prompts), len(benign_category_prompts), 30)
        
        # Use 5 samples per prompt as in the provided example
        num_samples = 5
        
        # Generate embeddings for harmful prompts
        harmful_embeddings = []
        for i in range(num_prompts):
            prompt = [prompts[i]] * num_samples
            text_input = tokenizer(prompt, padding="max_length", max_length=77, truncation=True, return_tensors="pt")
            with torch.no_grad():
                embed = text_encoder(text_input.input_ids.to('cuda'), return_dict=True)[0]
            harmful_embeddings.extend(embed.detach().cpu().numpy())
        
        harmful_embeddings = np.array(harmful_embeddings)
        
        # Generate embeddings for benign prompts
        benign_embeddings = []
        for i in range(num_prompts):
            prompt = [benign_category_prompts[i]] * num_samples
            text_input = tokenizer(prompt, padding="max_length", max_length=77, truncation=True, return_tensors="pt")
            with torch.no_grad():
                embed = text_encoder(text_input.input_ids.to('cuda'), return_dict=True)[0]
            benign_embeddings.extend(embed.detach().cpu().numpy())
        
        benign_embeddings = np.array(benign_embeddings)
        
        # Create concept vector
        concept_vector = np.mean(harmful_embeddings - benign_embeddings, axis=0)
        concept_vectors[category] = concept_vector
        
        # Save the concept vector
        np.save(os.path.join(output_dir, f"{category}_concept_vector.npy"), concept_vector)
        
    return concept_vectors

def main():
    # Define paths and parameters
    json_path = "jailbreak_diffusion_bench_filtered_400.json"
    model_dir = "CompVis/stable-diffusion-v1-4"
    output_dir = "data/concept_vectors"
    
    # Load data
    print("Loading benchmark data...")
    data = load_benchmark_data(json_path)
    
    # Group prompts by category
    print("Organizing prompts by category...")
    category_prompts = create_category_prompts(data)
    
    # Create benign prompts for each category
    print("Creating benign prompts...")
    benign_prompts = create_benign_prompts(category_prompts.keys())
    
    # Generate and save concept vectors
    print("Generating concept vectors...")
    concept_vectors = generate_concept_vectors(
        category_prompts, 
        benign_prompts, 
        model_dir, 
        output_dir
    )
    
    print(f"Concept vectors generated and saved to {output_dir}")
    # Print summary of categories and vector shapes
    for category, vector in concept_vectors.items():
        print(f"Category: {category}, Vector shape: {vector.shape}")

if __name__ == "__main__":
    main()

Loading benchmark data...
Organizing prompts by category...
Creating benign prompts...
Generating concept vectors...
Processing category: hate content
Processing category: discrimination and disinformation
Processing category: illegal activity
Processing category: physical harm and violence
Processing category: shocking
Processing category: harassment
Processing category: abuse
Processing category: nudity and sexual content
Concept vectors generated and saved to data/concept_vectors
Category: hate content, Vector shape: (77, 768)
Category: discrimination and disinformation, Vector shape: (77, 768)
Category: illegal activity, Vector shape: (77, 768)
Category: physical harm and violence, Vector shape: (77, 768)
Category: shocking, Vector shape: (77, 768)
Category: harassment, Vector shape: (77, 768)
Category: abuse, Vector shape: (77, 768)
Category: nudity and sexual content, Vector shape: (77, 768)
